In [ ]:
# CORE
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pickle
import joblib

# VISUALIZATION (EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# SCIKIT-LEARN PREPROCESSING
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize

# SCIKIT-LEARN MODEL SELECTION
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV

# SCIKIT-LEARN METRICS
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from sklearn.metrics.pairwise import cosine_similarity

# MACHINE LEARNING MODELS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# TENSORFLOW / KERAS
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks